In [1]:
# --- Heat Diffusion CA with Threshold Rule (PyCX style) ---
# States: integers 0..9  (0 = cold, 9 = hot)
#
#   Let A = average of k neighboring cells.
#   If A >= 7   → new state = 9
#   Else        → new state = round(A)

In [2]:
import matplotlib
matplotlib.use('TkAgg')
from pylab import *
import pycxsimulator

# ------------------ knobs ------------------
n      = 120       # grid size (n x n)
r      = 1         # Moore neighborhood radius
T_max  = 9         # maximum temperature (state 9)
# -------------------------------------------

In [3]:
def observe():
    global config, im
    im.set_data(config)

In [15]:
# Moore neighborhood
def update():
    global config, nextconfig

    for x in range(n):
        for y in range(n):

            # compute average of Moore neighbors (exclude self)
            s = 0.0
            cnt = 0
            for dx in range(-r, r+1):
                for dy in range(-r, r+1):
                    if dx == 0 and dy == 0:
                        continue  # skip self
                    xn = (x + dx) % n
                    yn = (y + dy) % n
                    s += config[xn, yn]
                    cnt += 1

            avg = s / cnt

            # threshold rule
            if avg >= 7.0:
                nextconfig[x, y] = 9.0
            else:
                nextconfig[x, y] = int(round(avg))

    # synchronous update swap
    config, nextconfig = nextconfig, config

In [17]:
# von Neumann neighborhood
def update():
    global config, nextconfig

    for x in range(n):
        for y in range(n):

            # von Neumann neighborhood: up, down, left, right
            s = 0.0
            cnt = 0
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                xn = (x + dx) % n
                yn = (y + dy) % n
                s += config[xn, yn]
                cnt += 1

            avg = s / cnt  # here cnt = 4

            # threshold rule
            if avg >= 7.0:
                nextconfig[x, y] = T_max
            else:
                nextconfig[x, y] = int(round(avg))

    # synchronous update swap
    config, nextconfig = nextconfig, config


In [10]:
# Initialization rule: uniform initial temperature = 5
def initialize():
    global config, nextconfig, im

    # uniform initial temperature = 5
    config = full((n, n), 5, dtype=int)

    # buffer for synchronous updates
    nextconfig = zeros((n, n), dtype=int)

    # draw initial frame (with only one colorbar)
    cla()
    im = imshow(config, vmin=0, vmax=T_max, cmap=cm.hot, interpolation='nearest')
    colorbar(label='temperature (0–9)')
    axis('off')
    title('Heat Diffusion CA (uniform init)')

pycxsimulator.GUI().start(func=[initialize, observe, update])

2025-11-12 22:23:34.907 python[80481:46755556] The class 'NSSavePanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


In [11]:
# Initialization rule: hot spot in the center

def initialize():
    global config, nextconfig, im

    # start all cells cold (0)
    config = zeros((n, n), dtype=int)

    # hot spot in the center
    cx, cy = n // 2, n // 2
    config[cx, cy] = T_max

    # buffer for synchronous update
    nextconfig = zeros((n, n), dtype=int)

    # draw initial frame (with only one colorbar)
    cla()
    im = imshow(config, vmin=0, vmax=T_max, cmap=cm.hot, interpolation='nearest')
    colorbar(label='temperature (0–9)')
    axis('off')
    title('Heat Diffusion CA (hot spot in center)')

pycxsimulator.GUI().start(func=[initialize, observe, update])


In [18]:
# Iniitialization rule: random temperature distribution 0-9

def initialize():
    global config, nextconfig, im

    # random temperatures from 0 to 9 (inclusive)
    config = randint(0, T_max + 1, size=(n, n))

    # buffer for synchronous update
    nextconfig = zeros((n, n), dtype=int)

    # draw initial frame (with only one colorbar)
    cla()
    im = imshow(config, vmin=0, vmax=T_max, cmap=cm.hot, interpolation='nearest')
    colorbar(label='temperature (0–9)')
    axis('off')
    title('Heat Diffusion CA (random init)')

pycxsimulator.GUI().start(func=[initialize, observe, update])
